# A brief summary of *GNU* Make
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Rules](#rules)
    * [The `.PHONY` Target](#the-phony-target)
    * [Pattern Rules](#pattern-rules)
        * [Static Pattern Rules](#static-pattern-rules)
    * [Built-in Rules](#built-in-rules)
    * [Wildcards](#wildcards)
    * [The `include` directive](#the-include-directive)
* [Variables](#variables)
    * [Automatic Variables](#automatic-variables)
    * [Built-in Variables](#built-in-variables)
    * [Command-line Variables](#command-line-variables)
    * [Target-specific Variables](#target-specific-variables)
    * [Conditional Directives](#conditional-directives)
* [Functions](#functions)
    * [String Manipulation Functions](#string-manipulation-functions)
        * [The `subst` function](#the-subst-function)
        * [The `patsubst` function](#the-patsubst-function)
        * [The `strip` function](#the-strip-function)
        * [The `findstring` function](#the-findstring-function)
        * [The `filter` function](#the-filter-function)
        * [The `sort` function](#the-sort-function)
        * [The `word` function](#the-word-function)
        * [The `wordlist` function](#the-wordlist-function)
        * [The `words` function](#the-words-function)
        * [The `firstword` function](#the-firstword-function)
    * [Filename functions](#filename-functions)
        * [The `dir` function](#the-dir-function)
        * [The `notdir` function](#the-notdir-function)
        * [The `suffix` function](#the-suffix-function)
        * [The `addsuffix` function](#the-addsuffix-function)
        * [The `join` function](#the-join-function)
        * [The `wildcard` function](#the-wildcard-function)
        * [The `realpath` function](#the-realpath-function)
    * [Conditional Functions](#conditional-functions)
        * [The `if` function](#the-if-function)
    * [Miscellaneous Functions](#miscellaneous-functions)
        * [The `foreach` function](#the-foreach-function)
        * [The `call` function](#the-call-function)
        * [The `shell` function](#the-shell-function)
        * [The `error` function](#the-error-function)
        

<hr>

*GNU Make* is a build-process automation tool, mainly employed in C/C++ projects.

It prevents the unnecessary re-compilation of many files, especially in a medium-to-large sized project, significantly reducing the compilation time.

The tool is called `make`, and by default, searches for a *Makefile* file in the working directory.

Alternatively, the file to be executed may be passed with the `-f` option.

## Rules <a class="anchor" id="rules"></a>

A *Makefile* consists, mainly, of a set of *rules*. Each rule specifies:
* A target file to generate.
* Prerequisite files that the target file depends on.
* Commands that use the prerequisite files to generate the target file.

In [ ]:
target: prerequisites
    command
    ...

When you run `$ make TARGET`,
* A rule is searched for, that matches the target file.
    * If not found and the target file does not exist, `$: No rule to make TARGET` is issued.
    * If not found and the target file exists, `$: Nothing to be done` is issued.
    * If found, `make` proceeds.
* Iteratively, a rule is searched for, that matches each prerequisite file.
    * If not found and the prerequisite file does not exist, `$: No rule to make PREREQUISITE needed by TARGET` is issued.
    * If not found and the prerequisite file exists, `make` proceeds.
    * If found, this procedure repeats recursively, with the prerequisite file as the target file.
* `make` evaluates whether to execute the associated rule commands, and generate the target file.
    * If the target file does not exist, the commands are executed.
    * If the target file exists, and the *Last Modified Timestamp*, in the file system, of the target file dates older than at least one prerequisite file, the commands are executed.
    * Otherwise, `$: Nothing to be done` is issued.

*Note:* By default, `$ make` assumes the target in the first rule in the *Makefile*.

*Note:* Preceed a command with `@` to stop `make` printing it again on the command-line. Alternatively, running `make` with the `-s` option places `@` before each command.

*Note:* Every command runs in a separate shell, in the background.

A multiple-target rule is permitted, and is equivalent to declaring multiple single-target rules, with the same prerequisites and commands.

In [ ]:
target-1 target-2 ...: prerequisites        # Multiple-target rule
    command
    ...

In [ ]:
target-1: prerequisites                     # Equivalent multiple single-target rule
    command
    ...

target-2: prerequisites
    command
    ...

...

A prerequisite file whose *Last Modified Timestamp* is ignored when `make` is evaluating whether to execute the associated rule commands, is placed after `|` in a rule, and is called an *order-only prerequisite*.

In [ ]:
target: prereqs | order-only-prereqs
    command
    ...

*Note:* An practical example of an *order-only* prerequisite is a directory, which may be a target file, though uncommon.

### The `.PHONY` Target <a class="anchor" id="the-phony-target"></a>

The `.PHONY` target is a built-in target in `make`, the prerequisites of which are targets themselves, and are called *phony targets*.

A phony target is not a target file, rather is treated as a command. Hence, when you run `$ make PHONY-TARGET`, the commands of a matching rule are always executed.

In [ ]:
.PHONY: phony-target-1 phony-target-2 ... 

phony-target-1: prerequisites
    command
    ...
    
phony-target-2: prerequisites
    command
    ...

A phony target may be a prerequisite to another rule. In that case, it is also treated as a command.

In [ ]:
.PHONY: phony-target

phony-target: prerequisites
    command
    ...

target: prerequisites phony-target
    command
    ...

### Pattern Rules <a class="anchor" id="pattern-rules"></a>

A *pattern rule* is a rule that captures targets with a common naming convention.

Usually, the common partition of the name, called *stem*, is the prefix of the target name, and is represented with `%`. The captured stem may then be used in the prerequisites.

In [ ]:
%.o: %.c                 # Captures all target object files, and 
    command              #   requires 'C' source files with a similar name
    ...

#### Static Pattern Rules <a class="anchor" id="static-pattern-rules"></a>

A *static pattern rule* is a pattern rule that applies to a specific list of target files only.

In [ ]:
A.o B.o C.o: %.o: %.c    # Pattern rule applies to 'A.o', 'B.o' and 'C.o' only
    command
    ...

### Built-in Rules <a class="anchor" id="built-in-rules"></a>

An *built-in rule* is a rule that `make` defines implicitly. For example,

In [ ]:
%.o: %.c                                # One of many implicit rules, defined by `make`
    $(CC) $(CPPFLAGS) $(CFLAGS) -c      # Variables, discussed later

*Note:* It is recommended to cancel all built-in rules, by passing the `-r` option to `make`.

### Wildcards <a class="anchor" id="wildcards"></a>

A *wildcard* is a special character(s), that may be placed within a target or prerequisite file names. It is expanded into all matching file names within a directory.
* `*` matches anything. For example, `*.c` expands into all *C* source files (within the working directory).
* `?` matches a single character. For example, `?.c` expands into all *C* source files with a single letter name.
* `[...]` matches a single character, of specific range.
    * `tmp[0-9].txt` expands into `tmp0.txt`, `tmp1.txt`, etc, if existing.
    * `file.[hc]` expands into `file.c` and `file.h`, if existing.
    * `[a-zA-Z].*` expands into all files with a single alphabetical character, and an extension.
    
*Note:* Wildcards may be used elsewhere using the `wildcard` function, discussed later.

### The `include` directive <a class="anchor" id="the-include-directive"></a>

The `include` directive may be used within a *Makefile* to import another.

In [ ]:
include $(CFG_FILE)

If the included *Makefile* matched a rule, it will be treated as a target file, and generated, before it is imported.

In [ ]:
$(CFG_FILE): prerequisites
    command
    ...

include $(CFG_FILE)

## Variables <a class="anchor" id="variables"></a>

A *variable* may be defined within a *Makefile* to store text, using `:=`.

In [ ]:
SRCS := A.c                # Assign 'A.c'
SRCS += B.c                # Append 'B.c'
SRCS := $(SRCS) C.c        # Append 'C.c'

.PHONY: all

all: $(SRCS)
    command
    ...

*Note:* `+=` may be used to append to a variable.

*Note:* Leading spaces are ignored in variable definitions, while trailing spaces are kept. This is usually insignificant, except in conditional directives, discussed later.

*Note:* Evaluating an undefined variable yields an empty string, and issues no error. Infact, a variable is considered undefined until it contains a non-empty string.

*Note:* It is recommended to define all variables before any rules.

### Automatic Variables <a class="anchor" id="automatic-variables"></a>

*Automatic variables* are a number of implicitly declared variables within any rule, that may be used within its commands only.
* `$@`, denotes the target file name.
* `$^`, denotes the names of all prerequisites (exclusive of order-only prerequisites).
* `$?`, denotes the names of all prerequisites, that are newer than the target.
* `$|`, denotes the names of all order-only prerequisites.
* `$*`, denotes the stem of a pattern rule.

In [ ]:
$(TARGET): $(SRCS)
    $(CC) $^ -o $@           # Uses automatic variables, for prerequisites and target

*Note:* `$(@F)`, `$(^F)`, and `$(?F)` evaluate to file names instead of complete paths.

*Note:* `$(@D)`, `$(^D)`, and `$(?D)` evaluate to containing directory paths instead of complete paths.

### Built-in Variables <a class="anchor" id="built-in-variables"></a>

A *built-in* variable is an implicitly-defined variable, by `make`.

| Name | Description |
| --- | --- |
| `MAKECMDGOALS` | The target(s) passed onto `make`, empty if implicitly determined. |

### Command-line Variables <a class="anchor" id="command-line-variables"></a>

A variable may be passed to `make` in the command-line,

`$ make CC:=GCC`

And evaluated, like any internally-defined variable, within a *Makefile*.

In [ ]:
main.exe: main.c
    $(CC) $^ -o $@           # CMD variable `CC` evaluated, like any internal variable

To override the value of a command-line variable, preceed its definition with the `override` directive.

In [ ]:
override CC := gcc

main.exe: main.c
    $(CC) $^ -o $@           # CMD variable `CC` evaluated, like any internal variable

### Target-specific Variables <a class="anchor" id="target-specific-variables"></a>

A variable may have different definitions, each for a different target.

In [ ]:
CFG := A                   # Default 'CFG' definition
%.o: CFG := B              # 'CFG' definition for any object file, as target
x.o: CFG := C              # 'CFG' definition of 'x.o', as target

*Note:* A target-specific definition is prioritized over a pattern-specific definition.

### Conditional directives <a class="anchor" id="conditional-directives"></a>

A *conditional directive* controls which lines, usually those that define variables, are included within a *Makefile*.

In [ ]:
conditional-directive-1
    VAR := VALUE-1
else conditional-directive-2
    VAR := VALUE-2
endif

There are four types of conditional directives in `make`.

In [ ]:
ifeq ($(A), $(B))          # True, if the two arguments are equal, inclusive of trailing spaces

In [ ]:
ifneq ($(A), $(B))         # Complements `ifeq`

In [ ]:
ifdef VAR                  # True, if 'VAR' is a non-empty variable

In [ ]:
ifndef VAR                 # Complements 'ifndef'

## Functions <a class="anchor" id="functions"></a>

A *function* is called using the following syntax.

In [ ]:
$(func-name arg1,arg2,...)

*Note:* Only leading trails of the first argument are ignored.

To use a comma within a function call, it must be wrapped in a variable. This works because splitting of arguments occurs after variable expansion.

Similarly, to define a single whitespace character, to be used in function calls, some trickery is employed.

In [ ]:
SRCS := x.c y.c z.c

comma := ,
empty :=
space := $(empty) $(empty)
SRCS := $(subst $(space),$(comma),$(SRCS))         # 'SRCS' contains 'x.c,y.c,z.c'

### String Manipulation Functions <a class="anchor" id="string-manipulation-functions"></a>

#### The `subst` function <a class="anchor" id="the-subst-function"></a>

`subst` replaces every occurence of `from` with `to` in `text`

In [ ]:
$(subst from,to,text)

#### The `patsubst` function <a class="anchor" id="the-patsubst-function"></a>

`patsubst` replaces every whitespace-separated word in `text`, matching `from`, with `to`, both of which may employ patterns.

In [ ]:
$(subst from,to,text)

In [ ]:
$(subst %.c,%.o,x.c y.c x.h y.h)        # yields 'x.o y.o x.h y.h'

A shorthand is defined for the most common use of `patsubst`, to replace suffixes (or, extensions).

In [ ]:
$(var:suffix=replacement)

In [ ]:
SRCS := x.c y.c z.c
OBJS := $(SRCS:.c=.h)                   # yields 'x.o y.o z.o' 

#### The `strip` function <a class="anchor" id="the-strip-function"></a>

`strip` removes leading and trailing whitespace from `string`, and replaces each internal sequence of whitespace characters with a single one.

In [ ]:
$(strip string)

In [ ]:
SRCS := x.c    y.c  z.c   
SRCS := $(strip $(SRCS))                # yields 'x.c y.c z.c'

#### The `findstring` function <a class="anchor" id="the-findstring-function"></a>

`findstring` searches for `find` in `in`. 
* If found, it yields `find`.
* If not found, it yields an empty string.

In [ ]:
$(findstring find,in)

#### The `filter` function <a class="anchor" id="the-filter-function"></a>

`filter` returns all whitespace-separated words in `text` that do match any of the pattern words.

In [ ]:
$(filter pattern,text)

In [ ]:
SRCS := x.c x.s x.h
SRCS := $(filter %.c %.s,$(SRCS))         # yields 'x.c x.s'

*Note:* The `filter-out` function complements the `filter` function.

#### The `sort` function <a class="anchor" id="the-sort-function"></a>

`sort` returns all whitespace-separated words in lexical order, removing duplicates.

In [ ]:
$(sort list)

In [ ]:
$(sort b a a c b)                            # yields 'a b c'

#### The `word` function <a class="anchor" id="the-word-function"></a>

`word` returns the *nth* whitespace-separated word, represented by `n` and using one-based indexing, in `text`.

In [ ]:
$(word n,text)

*Note:* If `n` exceeds the number of words in `text`, an empty string is returned.

#### The `wordlist` function <a class="anchor" id="the-wordlist-function"></a>

`wordlist` returns whitespace-separated words from text, from index `i` to `j`, one-based and all-inclusive.

In [ ]:
$(wordlist i,j,text)

#### The `words` function <a class="anchor" id="the-words-function"></a>

`words` returns the number of whitespace-separated words in `text`.

In [ ]:
$(words text)

#### The `firstword` function <a class="anchor" id="the-firstword-function"></a>

`firstword` returns the first whitespace-separated word in `text`.

In [ ]:
$(firstword text)

*Note:* The `lastword` function returns the last whitespace-separated word, instead.

### Filename Functions <a class="anchor" id="filename-functions"></a>

#### The `dir` function <a class="anchor" id="the-dir-function"></a>

`dir` returns the directory part of all files in a list of files.

In [ ]:
$(dir list)

*Note:* If a file contains no slashes, `./` is returned.

#### The `notdir` function <a class="anchor" id="the-notdir-function"></a>

`notdir` returns the file name of all files in a list of files.

In [ ]:
$(notdir list)

*Note:* If a file ends with a slash, it is replaced with an empty string.

#### The `suffix` function <a class="anchor" id="the-suffix-function"></a>

`suffix` extracts the suffix (or, extension) of each file in a list of files.

In [ ]:
$(suffix list)

*Note:* If a file has no extension, it is replaced with an empty string.

*Note:* The `basename` function extracts all, but the suffix of each file in a list of files.

#### The `addsuffix` function <a class="anchor" id="the-addsuffix-function"></a>

`addsuffix` appends `suffix` to each file in a list of files.

In [ ]:
$(addsuffix suffix,list)

In [ ]:
$(addsuffix .c,x y z)                   # yields 'x.c y.c z.c'

*Note:* The `addprefix` function appends `prefix` to each file in a list of files.

#### The `join` function <a class="anchor" id="the-join-function"></a>

`join` merges a list of whitespace-separated words, usually files, with another, pairwise.

In [ ]:
$(join list1,list2)

In [ ]:
$(join .c .h .s,x y z)                  # yields 'x.c y.h z.s'

In [ ]:
$(join .c .h,x y z)                     # yields 'x.c y.h z'

*Note:* If one list has more words than the other, the extra words are appended unchanged.

#### The `wildcard` function <a class="anchor" id="the-wildcard-function"></a>

`wildcard` allows the expansion of wildcards, within `text`, anywhere in a *Makefile*.

In [ ]:
$(wildcard text)

In [ ]:
$(wildcard *.c)                         # yields a list of all C source files, within the working directory

#### The `realpath` function <a class="anchor" id="the-realpath-function"></a>

`realpath` returns the real path of each file in a list of files.

In [ ]:
$(realpath list)

*Note:* The `abspath` function returns the absolute path of each file in a list of files.

*Note:* The absolute path of a shortcut is the shortcut's path, and the real path is the location of the file it points to. In most cases, however, the absolute and real paths for a file do not differ.

### Conditional Functions <a class="anchor" id="conditional-functions"></a>

#### The `if` function <a class="anchor" id="the-if-function"></a>

The `if` function evaluates to `then` if `condition` is not an empty string, otherwise `else`.

In [ ]:
$(if condition,then)                # If condition is an empty string, an empty string is returned

In [ ]:
$(if condition,then,else)           # If condition is an empty string,'else' is returned

*Note:* The `and` and `or` functions may be used as logical operators.

### Miscellaneous Functions <a class="anchor" id="miscellaneous-functions"></a>

#### The `foreach` function <a class="anchor" id="the-foreach-function"></a>

The `foreach` function assigns each whitespace-separated word in `list` to `var`, within every iteration, and evaluates `text` that may contain `var`.

In [ ]:
$(foreach var,list,text)

In [ ]:
$(foreach dir,x y z,$(wildcard dir/*.c))          # yields all C Source files within a list of directories

*Note:* Whitespace is placed between each of the non-empty strings that each iteration produces.

#### The `call` function <a class="anchor" id="the-call-function"></a>

The `call` function allows the calling of user-defined functions.
 
A user defined function uses the delayed-expansion assignment operator `=`, instead of `:=`. Additionally, special variables, that begin with `$(1)`, and count upwards, are used to denote function parameters.

In [ ]:
$(call function,arg1,arg2,...)

In [ ]:
swap = $(2) $(1)                        # 'swap' function: swaps two passed arguments
swapped := call(swap,A,B)               # yields 'B A'

*Note:* A missing argument becomes an empty string, while an extra argument is ignored.

#### The `shell` function <a class="anchor" id="the-shell-function"></a>

The `shell` function runs a command in a (separate) shell, and routes the standard output to replace the `shell` function call.

In [ ]:
$(shell command)

*Note:* The `shell` function replaces newline characters in the output with a single space.

#### The `error` function <a class="anchor" id="the-error-function"></a>

The `error` function prints an error message, before force-terminating `make`.

In [ ]:
$(error message)

*Note:* The `error` function, and all reporting functions, may be used as commands, or within conditional directives.

The `warning` function prints a warning message, without terminating.

In [ ]:
$(warning message)

The `info` function prints an informative message, and is equivalent to `echo message` in many shells.

In [ ]:
$(info message)